
# AAI612: Deep Learning & its Applications

*Notebook 5.3: KerasTuner*

<a href="https://colab.research.google.com/github/OmarMlaeb/AAI612_Malaeb/blob/master/Week%205/Notebook5.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ensure that KerasTuner is installed:

In [1]:
!pip install keras-tuner -q

Import KerasTuner and TensorFlow:

In [2]:
import keras_tuner
from tensorflow import keras
from tensorflow.keras import layers

`KerasTuner` lets you replace hard-coded hyperparameter values, such as units=32, with a range of possible choices, such as `Int(name="units", min_value=16, max_value=64, step=16)`. This set of choices in a given model is called the search space of the hyperparameter tuning process.  To specify a search space, define a model-building function (see the next listing).  It takes an hp argument, from which you can sample hyperparameter ranges, and it returns a compiled Keras model.

In [3]:
def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

**A KerasTuner `HyperModel`**

We can also adopt a more modular and configurable approach to model-building by subclassing the HyperModel class and define a build method, as follows:

In [4]:
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

The next step is to define a “tuner.” Schematically, you can think of a tuner as a for loop that will repeatedly
- Pick a set of hyperparameter values
- Call the model-building function with these values to create a model
- Train the model and record its metrics

KerasTuner has several built-in tuners available—RandomSearch, BayesianOptimization, and Hyperband. Let’s try BayesianOptimization, a tuner that attempts to make smart predictions for which new hyperparameter values are likely to perform best given the outcomes of previous choices:

In [5]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

Display an overview of the search space via `search_space_summary()`

In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [7]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 100 Complete [00h 00m 35s]
val_accuracy: 0.9741500020027161

Best val_accuracy So Far: 0.9765999913215637
Total elapsed time: 01h 32m 19s


## Querying the best hyperparameter configurations

Once the search is complete, you can query the best hyperparameter configurations, which you can use to create high-performing models that you can then retrain:

In [8]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

Before we can train on the full training data, though, there’s one last parameter we need to settle: the optimal number of epochs to train for. Typically, you’ll want to train the new models for longer than you did during the search: using an aggressive patience value in the EarlyStopping callback saves time during the search, but it may lead to under-fit models. Just use the validation set to find the best epoch:

In [9]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

Finally, train on the full dataset for just a bit longer than this epoch count, since
you’re training on more data; 20% more in this case:

In [10]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 1s 3ms/step - loss: 0.4199 - accuracy: 0.8892 - val_loss: 0.2393 - val_accuracy: 0.9337
Epoch 2/100
391/391 [==============================] - 1s 2ms/step - loss: 0.2122 - accuracy: 0.9399 - val_loss: 0.1774 - val_accuracy: 0.9496
Epoch 3/100
391/391 [==============================] - 1s 2ms/step - loss: 0.1655 - accuracy: 0.9518 - val_loss: 0.1554 - val_accuracy: 0.9554
Epoch 4/100
391/391 [==============================] - 1s 2ms/step - loss: 0.1371 - accuracy: 0.9605 - val_loss: 0.1380 - val_accuracy: 0.9613
Epoch 5/100
391/391 [==============================] - 1s 2ms/step - loss: 0.1168 - accuracy: 0.9661 - val_loss: 0.1215 - val_accuracy: 0.9646
Epoch 6/100
391/391 [==============================] - 1s 2ms/step - loss: 0.1016 - accuracy: 0.9705 - val_loss: 0.1187 - val_accuracy: 0.9656
Epoch 7/100
391/391 [==============================] - 1s 2ms/step - loss: 0.0892 - accuracy: 0.9746 - val_loss: 0.1108 - val_accuracy: 0.9691

In [12]:
best_models = tuner.get_best_models(top_n)